In [1]:
import bs4 , requests, re
import numpy as np
import pandas as pd
import datetime
import csv

In [2]:
today = str(datetime.datetime.now())[0:10]

In [3]:
class cricScrap():
    def __init__(self, min_page, max_page, scraped_date):
        self.scraped_date = scraped_date
        self.min_page = min_page
        self.max_page = max_page
        
        self.urls = self.generate_urls()
        self.soups = self.getHTML()
        self.cric_table = self.getCricData()
        
    def generate_urls(self):
        urls = []
        for i in range(self.min_page, self.max_page + 1):
            urls.append('https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;home_or_away=1;home_or_away=2;home_or_away=3;page={};template=results;type=team;view=results'.format(i))
        return urls
    
    def getHTML(self):
        soups = {}
        for url in self.urls:
            res = requests.get(url)
            soups[url] = bs4.BeautifulSoup(res.text)
        return soups
    def getCricData(self):
        fin_df = []
        for url, soup in self.soups.items():
            cric_vals = [x.text.split() for x in soup.find_all('tr', {'class' : 'data1'})]
            dfs = []
            for d in cric_vals:
                if len(d) == 10:
                    df_data = {
                    'team' : d[0],
                    'result' : d[1],
                    'margin' : np.nan,
                    'br' : np.nan,
                    'toss' : d[2],
                    'bat' : d[3],
                    'opposition' : d[5],
                    'ground' : d[6],
                    'date' : d[-1],
                    'url' : url,
                    'scrapped_date' : today
                }
                    df = pd.DataFrame([df_data])
                    dfs.append(df)
                    
                if len(d) == 12:
                    df_data = [{
                    'team' : d[0],
                    'result' : d[1],
                    'margin' : ' '.join(d[2:4]),
                    'br' : np.nan,
                    'toss' : d[4],
                    'bat' : d[5],
                    'opposition' :d[7],
                    'ground' : d[8],
                    'date' : d[-1],
                    'url' : url,
                    'scrapped_date' : today
                }]
                    df = pd.DataFrame(df_data)
                    dfs.append(df)

                if len(d) == 14:
                    df_data = [{
                    'team' : d[0],
                    'result' : d[1],
                    'margin' : ' '.join(d[2:4]),
                    'br' : d[4],
                    'toss' : d[5],
                    'bat' : d[6],
                    'opposition' : ' '.join(d[8:10]),
                    'ground' : d[-4],
                    'date' : d[-1],
                    'url' : url,
                    'scrapped_date' : today
                }]
                    df = pd.DataFrame(df_data)
                    dfs.append(df)
                    
                if len(d) == 15:
                    df_data = [{
                    'team' : d[0],
                    'result' : d[1],
                    'margin' : ' '.join(d[2:4]),
                    'br' : d[4],
                    'toss' : d[5],
                    'bat' : d[6],
                    'opposition' : ' '.join(d[8:10]),
                    'ground' : d[-4],
                    'date' : d[-1],
                    'url' : url,
                    'scrapped_date' : today
                }]
                    df = pd.DataFrame(df_data)
                    dfs.append(df)

                if len(d) == 13:
                    df_data = [{
                    'team' : d[0],
                    'result' : d[1],
                    'margin' : ' '.join(d[2:4]),
                    'br' : d[4],
                    'toss' : d[4],
                    'bat' : d[5],
                    'opposition' : ' '.join(d[8:9]),
                    'ground' : d[-4],
                    'date' : d[-1],
                    'url' : url,
                    'scrapped_date' : today
                }]
                    df = pd.DataFrame(df_data)
                    dfs.append(df)
            fin_df.append(pd.concat(dfs))
        return pd.concat(fin_df)
    

In [4]:
%time cric = cricScrap(min_page = 1, max_page = 178, scraped_date=today)

Wall time: 7min 21s


In [5]:
cric.cric_table

,team,result,margin,br,toss,bat,opposition,ground,date,url,scrapped_date
0,Australia,won,5 wickets,42,42,won,England,Melbourne,1971,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
0,England,lost,5 wickets,42,42,lost,Australia,Melbourne,1971,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
0,England,won,6 wickets,35,35,lost,Australia,Manchester,1972,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
0,Australia,lost,6 wickets,35,35,won,England,Manchester,1972,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
0,Australia,won,5 wickets,21,21,won,England,Lord's,1972,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
...,...,...,...,...,...,...,...,...,...,...,...
0,Ireland,lost,7 wickets,28,won,1st,Afghanistan Abu,Dhabi,2021,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
0,Bangladesh,won,120 runs,lost,lost,1st,Indies,Chattogram,2021,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
0,West,Indies,lost 120,runs,runs,won,Bangladesh,Chattogram,2021,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12
0,Afghanistan,won,36 runs,lost,lost,1st,Abu,Dhabi,2021,https://stats.espncricinfo.com/ci/engine/stats...,2021-02-12


In [9]:
cricdf = cric.cric_table

In [10]:
cricdf.to_csv('all_teams_cricket_data.csv', index = False)